In [ ]:
import os
import warnings
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.messages import HumanMessage, SystemMessage, BaseMessage
from IPython.display import Image, display
from typing import List, TypedDict


In [ ]:


# 환경 설정 로드 및 경고 무시
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])


In [ ]:



# Vector DB 초기화 (카페 메뉴 DB)
embeddings_model = OllamaEmbeddings(model="bge-m3:latest")
cafe_db = FAISS.load_local(
    "./db/cafe_db",
    embeddings_model,
    allow_dangerous_deserialization=True
)

# 1단계: 카페 메뉴 검색 도구 정의
@tool
def search_cafe_menu(query: str) -> str:
    """카페 메뉴에서 정보를 검색합니다. 고객의 질문과 관련된 메뉴를 찾아 상세 정보를 제공합니다."""
    docs = cafe_db.similarity_search(query, k=6)

    formatted_docs = "\n\n---\n\n".join([
        f'<Document source="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
        for doc in docs
    ])

    if len(docs) > 0:
        return formatted_docs
    return "관련 카페 메뉴 정보를 찾을 수 없습니다."

# 2단계: 에이전트 상태 정의 (TypedDict 사용)
class AgentState(TypedDict):
    """
    ReAct Agent의 현재 상태를 나타내는 클래스.
    메시지 목록을 관리하여 대화의 흐름을 추적합니다.
    """
    messages: List[BaseMessage]

# 3단계: LLM과 도구 설정
# LLM 모델 초기화 (Groq API를 사용하여 Llama 3 모델 사용)
llm = ChatOpenAI(
    # api_key=os.getenv("OPENAI_API_KEY"), # Groq 사용 시 주석 처리
    base_url="https://api.groq.com/openai/v1",
    model="llama3-8b-8192",  # 또는 "llama3-70b-8192" 등 적절한 Groq 모델 선택
    temperature=0.7,
    # api_key=os.getenv("GROQ_API_KEY") # Groq API 키를 여기에 직접 전달할 수도 있습니다.
)

# LLM에 도구 바인딩
tools = [search_cafe_menu]
llm_with_tools = llm.bind_tools(tools=tools)

# 4단계: 시스템 프롬프트 정의
system_prompt = """
당신은 카페의 친절한 AI 어시스턴트입니다. 고객의 메뉴 관련 질문에 정확하고 상세하게 답변해주세요.
필요한 경우, 'search_cafe_menu' 도구를 사용하여 카페 메뉴 데이터베이스에서 정보를 찾아 답변에 활용하세요.
고객의 질문을 명확히 이해하고, 최적의 정보를 제공하도록 노력하세요.
"""

# 5단계: 노드 함수 정의
def agent_node(state: AgentState):
    """
    LLM을 호출하여 사용자의 질문에 대한 응답을 생성하거나,
    필요한 도구 호출을 결정하는 핵심 에이전트 노드입니다.
    """
    print("\n--- Agent 노드: LLM 호출 시작 ---")

    # 시스템 메시지를 대화의 시작에 추가
    messages = [SystemMessage(content=system_prompt)] + state['messages']
    
    # LLM 호출
    response = llm_with_tools.invoke(messages)
    
    print(f"--- LLM 응답 생성 완료. 도구 호출 여부: {bool(response.tool_calls)} ---")
    
    return {"messages": [response]}

# 6단계: LangGraph 그래프 구성
builder = StateGraph(AgentState)

# 노드 추가: 'agent' (LLM 호출) 및 'tools' (도구 실행)
builder.add_node("agent", agent_node)
builder.add_node("tools", ToolNode(tools))

# 시작 지점 설정
builder.set_entry_point("agent")

# 조건부 엣지: 'agent' 노드의 응답에 따라 다음 노드 결정
# tools_condition은 응답에 tool_calls가 있으면 'tools'로, 없으면 'END'로 자동 전환
builder.add_conditional_edges(
    "agent",
    tools_condition,  # LLM 응답에 따라 자동으로 도구 호출 여부 판단
    {"tools": "tools", END: END} # 도구 호출이 필요하면 'tools' 노드로, 아니면 그래프 종료
)

# 'tools' 노드 실행 후 다시 'agent' 노드로 돌아가서 LLM이 도구 결과를 바탕으로 최종 답변 생성
builder.add_edge("tools", "agent")

# 7단계: 그래프 컴파일
custom_react_agent = builder.compile()

print("LangGraph 기반의 사용자 정의 ReAct Agent가 성공적으로 생성되었습니다!")
print(f"Agent 타입: {type(custom_react_agent)}")



In [ ]:


# 8단계: 그래프 시각화 (선택 사항)
try:
    display(Image(custom_react_agent.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"그래프 시각화 중 오류 발생: {e}. 시각화를 건너뜜.")
    


In [ ]:

# 9단계: Agent 테스트 함수
def run_agent_test(question: str, agent_instance):
    """주어진 질문으로 에이전트를 실행하고 결과 과정을 출력합니다."""
    print(f"\n{'='*60}")
    print(f"질문: {question}")
    print('='*60)

    # Agent 실행
    inputs = {"messages": [HumanMessage(content=question)]}
    result = agent_instance.invoke(inputs)

    # 실행 과정 출력
    print("\n=== 실행 과정 ===")
    for i, message in enumerate(result['messages']):
        if hasattr(message, 'tool_calls') and message.tool_calls:
            print(f"{i+1}. {type(message).__name__}: 도구 호출 - {message.tool_calls[0]['name']} (쿼리: '{message.tool_calls[0]['args']['query']}')")
        else:
            content_preview = message.content[:100] + "..." if len(message.content) > 100 else message.content
            print(f"{i+1}. {type(message).__name__}: {content_preview}")

    # 최종 답변 출력
    print(f"\n=== 최종 답변 ===")
    print(result['messages'][-1].content)

# 10단계: 테스트 실행
if __name__ == "__main__":
    test_questions = [
        "아메리카노와 아이스 아메리카노의 차이점과 가격을 알려주세요.",
        "라떼 종류에는 어떤 메뉴들이 있고 각각의 특징은 무엇인가요?",
        "디저트 메뉴 중에서 티라미수에 대해 자세히 설명해주세요."
    ]

    # 단일 테스트 (코드의 메시지 흐름 분석을 위해 남겨둠)
    print("\n=== 단일 테스트 (자세한 메시지 흐름) ===")
    single_test_question = "콜드브루와 아이스 아메리카노의 차이점이 무엇인가요?"
    run_agent_test(single_test_question, custom_react_agent)

    # 전체 테스트
    print("\n=== 전체 테스트 실행 ===")
    for q in test_questions:
        run_agent_test(q, custom_react_agent)